## Example - BCDI

In [1]:
import networkx as nx
import sys
import numpy as np
from abc import ABC, abstractmethod
from collections import namedtuple
from copy import deepcopy
from functools import reduce
from typing import Callable, Any
from itertools import accumulate

In [2]:
from systemflow.node import *
from systemflow.auxtypes import is_proportion


In this example, we'll define primitives and functions needed to begin modeling inverse-space imaging techniques such as Bragg Coherent Diffraction Imaging (BCDI).

We'll start with a sample stage, which has several variables that allow position and movement to be used to construct a latency - this can be used in more advanced examples to determine the productivity improvements for intelligent scanning and positioning over a grid search.

In [3]:
# these and other X-ray related SystemFlow modules are located in the xrs.py
# source file
from systemflow.xrs import *

In [4]:
# the introduction of sample-based properties is accomplished via
# the PositionSample mutation
ps = PositionSample()

<function PositionSample.<lambda> at 0x1096e6b90>


In [5]:
# the sample_stage is the host component producing these mutations and
# controlled by a variety of parameters:
sample_stage = Component("Sample Stage",
                    [PositionSample(),],
                    {"position (mm,mm)": [0.0, 0.0],
                     "last position (mm,mm)": [0.0, 0.0],
                     "move rate (mm/s)": 100,
                     "settle time (s)": 1e-3,},
                     {})

<function PositionSample.<lambda> at 0x1096e6b90>


In [6]:
# to test - construct an empty message
msg0 = Message({}, {})
# and pass it through the mutation hosted on sample stage
msg1, _ = ps(msg0, sample_stage)

Position:  [0.0, 0.0]
Relevancy: 1.0


In [7]:
# we see that the described fields and properties in the message have been created
msg1

Message(fields={'relevancy (%)': 1.0, 'position (mm,mm)': [0.0, 0.0], 'movement latency (s)': np.float64(0.001)}, properties={})

After positioning the sample, the X-ray light incident on it will diffract and be collected by a two-dimensional sensor. The introduction of this information into the system is done via CollectImage. We declare the mutation, its host component, and parameters:

In [8]:
ci = CollectImage()

In [9]:
vc_img = collect_parameters([CollectImage()])

In [10]:
vc_img

VarCollection(
    resolution='resolution (n,n)',
    bitdepth='bit depth (n)',
    readout='readout latency (s)',
    pixelenergy='pixel energy (J)',
    sample_rate='sample rate (Hz)'
)

In [11]:
ci_host = Component("Image sensor",
                    [CollectImage(),],
                    parameters = {vc_img.resolution: (2000, 2000),
                     vc_img.bitdepth: 16,
                     vc_img.readout: 1e-3,
                     vc_img.pixelenergy: 1e-3,
                     vc_img.sample_rate: 10e3,})

In [12]:
msg2, _ = ci(msg1, ci_host)

In [13]:
msg2

Message(fields={'relevancy (%)': 1.0, 'position (mm,mm)': [0.0, 0.0], 'movement latency (s)': np.float64(0.001), 'image data (B)': np.float64(8000000.0), 'readout latency (s)': 0.001}, properties={'resolution (n,n)': (2000, 2000), 'bitdepth (n)': 16, 'sample rate (Hz)': 10000.0, 'images (n)': 1})

Next is a flat-field correction, which models applying a constant offset and/or scaling factor across the input image:

In [14]:
ffc = FlatFieldCorrection()

In [15]:
vc = collect_parameters([ffc])

In [16]:
vc

VarCollection(
    op_latency='op latency (s)',
    parallelism='parallelism (%)'
)

In [17]:
ffc_host = Component("Preprocessor 1",
                    [FlatFieldCorrection(),],
                    parameters = {vc.op_latency: 1e-6,
                                  vc.parallelism: 0.75,})

In [18]:
msg3, _ = ffc(msg2, ffc_host)

This is followed by interpolating mask correction, which "fills in" data missing as a result of dead/saturated pixels or a gap between sensor modules. We assume a nearest-neighbor kernel-based interpolation is being used, with a size controllable by percentage of the original image used for the kernel.

In [19]:
mc = MaskCorrection()

In [20]:
vc = collect_parameters([mc])

In [21]:
vc

VarCollection(
    mask_proportion='masking proportion (%)',
    op_latency='op latency (s)',
    parallelism='parallelism (%)',
    kernel_size='kernel size (%,%)'
)

In [22]:
mc_host = Component("Preprocessor 2",
                    [MaskCorrection(),],
                    parameters = {vc.op_latency: 1e-6,
                                  vc.parallelism: 0.75,
                                  vc.kernel_size: (0.02, 0.02), # how big is the interpolation kernel?
                                  vc.mask_proportion: 0.05,}) # how much of the image is under the mask?

In [23]:
msg3.properties

{'resolution (n,n)': (2000, 2000),
 'bitdepth (n)': 16,
 'sample rate (Hz)': 10000.0,
 'images (n)': 1}

In [24]:
msg4, h2 = mc(msg3, mc_host)

In [25]:
h2

{'masking operations (n,n)': (np.float64(133.7480609952844),
  np.float64(2392558.049953953))}

In [26]:
msg4.fields

{'relevancy (%)': 1.0,
 'position (mm,mm)': [0.0, 0.0],
 'movement latency (s)': np.float64(0.001),
 'image data (B)': np.float64(8000000.0),
 'readout latency (s)': 0.001,
 'flatfield latency (s)': np.float64(4.4721359549995795e-05),
 'masking corrections (B)': np.float64(400000.0),
 'masking latency (s)': np.float64(0.0001337480609952844)}

These pre-processing steps are followed by phase reconstruction. We model this as an iterative algorithm: given an initial "guess" of the real-space image, a Fourier transform is taken and compared to the measured diffraction pattern. The error between these two images is measured and descended on to improve the reconstruction. Many (10+) steps are generally taken to converge on a satisfactory reconstruction.

In [27]:
pr = PhaseReconstruction2D()

In [28]:
vc = collect_parameters([pr])

In [29]:
vc

VarCollection(
    op_latency='op latency (s)',
    parallelism='parallelism (%)',
    iterations='iterations (n)'
)

In [30]:
phase_host = Component("Processor",
                    [PhaseReconstruction2D(),],
                    parameters = {vc.parallelism: 0.75,
                                  vc.op_latency: 1e-7,
                                  vc.iterations: 20,})

In [31]:
msg5, _ = pr(msg4, phase_host)

In [32]:
msg5.fields

{'relevancy (%)': 1.0,
 'position (mm,mm)': [0.0, 0.0],
 'movement latency (s)': np.float64(0.001),
 'image data (B)': np.float64(8000000.0),
 'readout latency (s)': 0.001,
 'flatfield latency (s)': np.float64(4.4721359549995795e-05),
 'masking corrections (B)': np.float64(400000.0),
 'masking latency (s)': np.float64(0.0001337480609952844),
 'phase data (B)': 64000000}

In [33]:
msg5.properties

{'resolution (n,n)': (2000, 2000),
 'bitdepth (n)': 16,
 'sample rate (Hz)': 10000.0,
 'images (n)': 1,
 'phase reconstruction (n,n)': (2000, 2000),
 'phase reconstruction latency (s)': np.float64(1.5703820405451533e-05)}

Now, we'll assemble each of these steps into a single system which we can evaluate. We'll also combine the flat-field correction, masking, and phase reconstruction onto a single "CPU" host:

In [34]:
cpu_mutations = [FlatFieldCorrection(),
                     MaskCorrection(),
                     PhaseReconstruction2D(),]

vc = collect_parameters(cpu_mutations)
cpu_host = Component("Processor",
                    cpu_mutations,
                    parameters = {vc.op_latency: 1e-6,
                                  vc.parallelism: 0.75,
                                  vc.kernel_size: (0.02, 0.02),
                                  vc.mask_proportion: 0.05,
                                  vc.op_latency: 1e-7,
                                  vc.iterations: 20,})

In [35]:
nodes = [sample_stage, ci_host, cpu_host]

In [36]:
links = [DefaultLink("Sample Stage -> Image sensor",
                     "Sample Stage",
                     "Image sensor"),
        DefaultLink("Image sensor -> Processor",
                     "Image sensor",
                     "Processor"),]

We'll also define metrics that look at the main host properties predicted by the ExecutionGraph which we want to measure:

In [37]:
class ReconstructionPower(Metric):
    def __init__(self):
        super().__init__("Phase reconstruction power", 
                         [],
                         [PhaseReconstruction2D().outputs.host_properties.ops],)
        
    def metric(self, message: Message, properties: dict):
        matches = self.graph_matches(properties)
        power = np.prod(matches[0]) * 1e-8
        metrics = {"reconstruction power (W)": power,}
        
        return metrics
    

In [38]:
class TotalOps(Metric):
    def __init__(self):
        super().__init__("Total operations", 
                         [],
                         [Regex(r"ops \(n,n\)"),],)
        
    def metric(self, message: Message, properties: dict):
        matches = self.graph_matches(properties)
        ops = np.sum([np.prod(op) for op in matches])
        metrics = {"total ops (n)": ops,}
        
        return metrics
    

In [39]:
class TotalLatency(Metric):
    def __init__(self):
        super().__init__("Total latency", 
                         [Regex(r"latency \(s\)"),],
                         [],)
        
    def metric(self, message: Message, properties: dict):
        matches = self.message_matches(message)
        ops = np.sum(matches)
        metrics = {"total latency (s)": ops,}
        
        return metrics
    

In [40]:
bcdi_graph = ExecutionGraph("BCDI Experiment", nodes, links, [ReconstructionPower(), TotalOps(), TotalLatency()])

In [41]:
g2 = bcdi_graph()

Position:  [0.0, 0.0]
Relevancy: 1.0


In [42]:
g2

In [43]:
g2.metric_values

{'reconstruction power (W)': np.float64(6.08164799306237),
 'total ops (n)': np.float64(612164799.306237),
 'total latency (s)': np.float64(0.0020335507624599796)}

In [44]:
g2.get_all_node_parameters()

{'Processor': {'op latency (s)': 1e-07,
  'parallelism (%)': 0.75,
  'kernel size (%,%)': (0.02, 0.02),
  'masking proportion (%)': 0.05,
  'iterations (n)': 20},
 'Image sensor': {'resolution (n,n)': (2000, 2000),
  'bit depth (n)': 16,
  'readout latency (s)': 0.001,
  'pixel energy (J)': 0.001,
  'sample rate (Hz)': 10000.0},
 'Sample Stage': {'position (mm,mm)': [0.0, 0.0],
  'last position (mm,mm)': [0.0, 0.0],
  'move rate (mm/s)': 100,
  'settle time (s)': 0.001}}

And we'll setup one experiment to sweep over the sensor resolution:

In [45]:
def sweep_resolution(resolution: tuple, exg: ExecutionGraph):
    # an empirical relationship we assume between the classifier skill and number of filters
    new_params = {"Image sensor": {vc_img.resolution: resolution,}}
    # we can simply call an existing graph with new parameters
    new_exg = exg.with_updated_parameters(new_params)()

    power = new_exg.metric_values["reconstruction power (W)"]
    ops = new_exg.metric_values["total ops (n)"]
    latency = new_exg.metric_values["total latency (s)"]
    return power, ops, latency

In [46]:
# assume aspect ratio stays the same and scales up and down
resolutions = [np.astype(s * np.array((2000, 1400)), 'int') for s in np.linspace(start=0.8, stop=6.0, num=101)]
megapixels = [np.prod(r)/1e6 for r in resolutions]

In [47]:
metrics = [sweep_resolution(r, g2) for r in resolutions]

Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
Position:  [0.0, 0.0]
Relevancy: 1.0
P

In [48]:
import plotly.graph_objects as go

Now, we can predict the overall power and latency for a single image by number of megapixels: power goes up linearly, and with a parallel implementation of the algorithms, latency goes up sub-linearly:

In [49]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = megapixels,
    y = [m[0] for m in metrics],))

fig.update_layout(
    title_text = "Total Processing Power by Sensor Pixels",
    xaxis_title="Sensor Pixels (MP)",
    yaxis_title="Total Power (W)",
)
fig.show()

In [50]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = megapixels,
    y = [m[2] for m in metrics],
    name = "latency (s)",))

fig.update_layout(
    title_text = "Total Processing Latency by Sensor Pixels",
    xaxis_title="Sensor Pixels (MP)",
    yaxis_title="Total Latency (s)",
)
fig.show()

We'll expand on this functionality in the case where many images are used to do the reconstruction, as is the case in ptychography, tomography, and laminography.